# Mass flow conservation

Mass flow conservation is a key principle for steady flows. Though it is defined as an integral sum, it is widely used with averaged values. The surface is supposed to be normal the velocity. In this context,
$\dot{m}=\rho V S$ can be expanded as

$$ 
\dot{m}=\rho S V  = \frac{S P_t}{\sqrt{rT_t}} \dot{m_R}(M)
$$

where the weighted mass flow function

$$
\dot{m_R}(M) = \sqrt\gamma M \left( 1+\frac{\gamma-1}{2}M^2 \right)^{-\frac{\gamma+1}{2(\gamma-1)}} 
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

gam = 1.4

def machplot(ax, name, value):
    global mach
    ax.grid()
    ax.set_xlim(0., mach.max())
    ax.set_xlabel('Mach')
    ax.set_ylabel(name)
    ax.plot(mach, value)

def plot_arrow(ax, xyfrom, xyto, color='red'):
    from matplotlib.patches import FancyArrowPatch
    arrow = FancyArrowPatch(xyfrom, xyto,
        mutation_scale=20,  
        color=color, alpha=0.4,
        linewidth=1)
    ax.add_patch(arrow)
    ax.plot(*xyfrom, marker='o', color='black', ms=5)


In [ ]:
def mr(mach):
    return np.sqrt(gam) * mach * ( 1 + .5*(gam-1)*mach**2 )** (-(gam+1)/2/(gam-1))

def sigma(mach):
    return mr(1.)/mr(mach)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10,4))
mach = np.linspace(.05, 4., 100)

# above defined plot
machplot(ax0, "$\dot{m_R}(M)$", mr(mach))
machplot(ax1, "$\Sigma(M)$", sigma(mach))

## Variation of section

In a diverging duct, one can compute the new Mach number $M_2$ from the measured initial Mach number $M_1$ and the section ratio $A_2/A_2=1.2$. Two cases are computed below:

- an initial subsonic $M_1=0.4$ flow where the divergence implies an acceleration (blue)

- an initial supersonic $M_1=1.4$ flow where the divergence implies an deceleration (red)


`````{admonition} This admonition was styled...
:class: tip
With a tip class!
`````


In [ ]:
section_ratio = 1.6

def mach_from_sigma(sigmavalue, mach_init):
    from scipy.optimize import newton
    return newton(lambda mach: sigmavalue - sigma(mach), # function to solve
                  mach_init)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10,4))
mach = np.linspace(.1, 3., 100)
machplot(ax0, "$\dot{m_R}(M)$", mr(mach))
machplot(ax1, "$\Sigma(M)$", sigma(mach)) ; ax1.set_ylim(0, 3)

print(f"The Mach number obtained after a ratio {section_ratio} of section:")
for m1, color in ((0.5, 'blue'), (1.5, 'red')):
    # ACTUAL computation to proceed 
    sig1 = sigma(m1)
    sig2 = section_ratio * sig1
    m2 = mach_from_sigma(sig2, m1) # initialisation with m1
    # weighted mass flows functions
    mr1 = mr(1.)/sig1
    mr2 = mr(1.)/sig2
    plot_arrow(ax0, (m1, mr1), (m2, mr2), color)
    plot_arrow(ax1, (m1, sig1), (m2, sig2), color)
    print(f"is {m2:.2f} if the initial Mach number is {m1:.1f} ({color})")


```{admonition} Multiple solutions for an expected mass flow
Here's the admonition content
```

## Multiple solutions

From a given value of mass flow of $\Sigma$ function, there are two solutions for the Mach number, one subsonic and the other supersonic. If mass flow conservation is applied to close sections, the Mach number is assumed to be in the same regime. However, a global conservation can be applied between two sections of different regimes. The selection of the regime can be made with some strategy assumptions and pressure compatibility relations. Those considerations are detailed in {doc}`../internal/nozzle-regime-details`. 

Below, the computation and the identification of both solution are provided.

In [ ]:
# base figure
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10,4))
mach = np.linspace(.1, 3., 100)
machplot(ax0, "$\dot{m_R}(M)$", mr(mach))
machplot(ax1, "$\Sigma(M)$", sigma(mach)) ; ax1.set_ylim(0, 3)
#
m1 = .5
sig1 = sigma(m1)
sig2 = section_ratio * sig1
m2sub = mach_from_sigma(sig2, .5) # initialisation with subsonic
m2sup = mach_from_sigma(sig2, 1.5) # initialisation with supersonic
# weighted mass flows functions
mr1 = mr(1.)/sig1
mr2 = mr(1.)/sig2
for m2 in (m2sub, m2sup): # plot all arrows
    plot_arrow(ax0, (m1, mr1), (m2, mr2), 'blue')
    plot_arrow(ax1, (m1, sig1), (m2, sig2), 'blue')


All provided functions are implemented and available in (aerokit python package)[http://github.com/jgressier/aerokit] in `aero.massflow` subpackage.